In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('jobs_dataset.csv')
display(df.head())
list(df.columns)


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


['timestamp',
 'company',
 'level',
 'title',
 'totalyearlycompensation',
 'location',
 'yearsofexperience',
 'yearsatcompany',
 'tag',
 'basesalary',
 'stockgrantvalue',
 'bonus',
 'gender',
 'otherdetails',
 'cityid',
 'dmaid',
 'rowNumber',
 'Masters_Degree',
 'Bachelors_Degree',
 'Doctorate_Degree',
 'Highschool',
 'Some_College',
 'Race_Asian',
 'Race_White',
 'Race_Two_Or_More',
 'Race_Black',
 'Race_Hispanic',
 'Race',
 'Education']

In [15]:
df = df[['company', 'totalyearlycompensation', 'location',
        'yearsofexperience', 'basesalary', 'stockgrantvalue', 'bonus', 
        'cityid']]
display(df)

,company,totalyearlycompensation,location,yearsofexperience,basesalary,stockgrantvalue,bonus,cityid
0,Oracle,127000,"Redwood City, CA",1.5,107000.0,20000.0,10000.0,7392
1,eBay,100000,"San Francisco, CA",5.0,0.0,0.0,0.0,7419
2,Amazon,310000,"Seattle, WA",8.0,155000.0,0.0,0.0,11527
3,Apple,372000,"Sunnyvale, CA",7.0,157000.0,180000.0,35000.0,7472
4,Microsoft,157000,"Mountain View, CA",5.0,0.0,0.0,0.0,7322
...,...,...,...,...,...,...,...,...
62637,Google,327000,"Seattle, WA",10.0,155000.0,150000.0,22000.0,11527
62638,Microsoft,237000,"Redmond, WA",2.0,146900.0,73200.0,16000.0,11521
62639,MSFT,220000,"Seattle, WA",14.0,157000.0,25000.0,20000.0,11527
62640,Salesforce,280000,"San Francisco, CA",8.0,194688.0,57000.0,29000.0,7419


In [ ]:
df.describe()